In [1]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [16]:
import tensorflow as tf
import numpy as np
from tensorflow.python.client import device_lib
import tensorflow.contrib.immediate as immediate

device_names = [d.name for d in device_lib.list_local_devices()]
print device_names

[u'/cpu:0']


ImportError: No module named PIL

README.md           lbfgs-macbook.ipynb mnist4
data                logs                mnist4.zip
dataset-mnist.lua   mnist.t7            port.lua


## Convert MNIST csv data to numpy

In [4]:
data = np.loadtxt("mnist.t7/test_32x32.t7.csv.data", dtype=np.uint8).reshape(10000, 32, 32)
np.save("mnist.t7/test_32x32.npy", data)

In [6]:
data = np.loadtxt("mnist.t7/train_32x32.t7.csv.data", dtype=np.uint8).reshape(60000, 32, 32)
np.save("mnist.t7/train_32x32.npy", data)

In [8]:
data = np.loadtxt("mnist.t7/train_32x32.t7.csv.labels", dtype=np.uint8)
np.save("mnist.t7/train_labels.npy", data)

In [9]:
data = np.loadtxt("mnist.t7/test_32x32.t7.csv.labels", dtype=np.uint8)
np.save("mnist.t7/test_labels.npy", data)

## Reproduce loss on first example

In [11]:
train_data = np.load("mnist.t7/train_32x32.npy")

In [47]:
train_labels = np.load("mnist.t7/train_labels.npy")

In [32]:
env = immediate.Env(tf)
tfi = env.tf

In [74]:
i = 0
W = tfi.ones((10, 1024), dtype=tf.float32)
b = tfi.ones((10, 1), dtype=tf.float32)
x = env.numpy_to_itensor(train_data[i], dtype=tf.float32)
x_flat = tfi.reshape(x, (1024, 1))
logits = tfi.matmul(W, x_flat)+b
targets = tfi.one_hot(train_labels[i]-1, 10)  # Lua labels are off-by-one
targets = tfi.reshape(targets, (1, 10))
logits = tfi.reshape(logits, (1, 10))
cross_entropy = tfi.nn.softmax_cross_entropy_with_logits(logits, targets)

In [75]:
cross_entropy

ITensor([ 2.30258512], dtype=float32)

In [65]:
a = env.numpy_to_itensor([[0,0,0,1]], dtype=tf.float32)
b = env.numpy_to_itensor([[1,1,1,1]], dtype=tf.float32)
tfi.nn.softmax_cross_entropy_with_logits(b*100, a)

ITensor([ 1.38629436], dtype=float32)

In [22]:
x = train_data[0].reshape((1024))

In [26]:
np.dot(x, W)+b

array([[ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  27526.,  27526.,  27526.,  27526.,
         27526.,  27526.,  27526.],
       [ 27526.,  27526.,  27526.,  2

# Scratch

In [27]:
import tensorflow as tf
nInputs = 6
nUnitsLayer1 = 3
weights = tf.Variable(tf.truncated_normal([nInputs, nUnitsLayer1]))
biases= tf.Variable(tf.zeros(nUnitsLayer1))

print 'nInputs', nInputs
print 'nUnitsLayer1', nUnitsLayer1
print 'weights shape', tf.shape(weights)
print 'biases shape', tf.shape(biases)

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    print 'weights'
    print (weights.eval())
    print 'biases'
    print (biases.eval())


nInputs 6
nUnitsLayer1 3
weights shape Tensor("Shape:0", shape=(2,), dtype=int32)
biases shape Tensor("Shape_1:0", shape=(1,), dtype=int32)
weights
[[-0.42564794  1.32690036 -0.57812071]
 [ 1.3026278  -1.29796839 -1.02938724]
 [ 0.19238499  1.10781586 -0.24808854]
 [-0.60994369 -0.30048054  0.5742262 ]
 [ 0.13138418 -0.60150367  0.44378978]
 [ 0.42674765 -0.44903693  0.32681581]]
biases
[ 0.  0.  0.]


In [28]:
weights.get_shape()

TensorShape([Dimension(6), Dimension(3)])

In [29]:
weights

In [31]:
tf.shape(weights).get_shape

TensorShape([Dimension(2)])